###### Project

In [233]:
import pandas as pd
#import sqlite3
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import datetime as dt
import folium
from folium import plugins
import geopandas
from shapely.geometry.polygon import LinearRing, Polygon

spray_data = pd.read_csv(
    'Files/spray.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

train_data = pd.read_csv(
    'Files/train.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

weather_data = pd.read_csv(
    'Files/weather.csv',  # шлях до файлу, який містить дані
    sep=',',  # роздільник стовпців
    header=0  # номер рядка, що містить заголовок (якщо заголовка немає, можна передати None)
)

test_truncated_data = pd.read_excel(
    'Files/test_truncated.xlsx',
    sheet_name='test_truncated',  # прочитати всі листи
    usecols="A:J"
    #nrows=5
)

'''DataSets={1:spray_data,
         2:train_data,
         3:weather_data,
         4:test_truncated_data}
'''

#test_truncated_data.head()
#dates=spray_data [['Date'],['Time']]
#train_data.groupby(train_data["Address"])
#dir(spray_data)
#spray_data.amin(axes='Latitude')
#test_truncated_data.info()
#train_data
#weather_data

'DataSets={1:spray_data,\n         2:train_data,\n         3:weather_data,\n         4:test_truncated_data}\n'

In [234]:
#spray_data_polygon=spray_data.copy()

#spray_data['hour']=pd.to_datetime(spray_data.Time).dt.day
#spray_data['minute']=pd.to_datetime(spray_data.Time).dt.weekday
#spray_data['day']=pd.to_datetime(spray_data.Date).dt.day
#spray_data['weekday']=pd.to_datetime(spray_data.Date).dt.weekday
spray_data['month']=pd.to_datetime(spray_data.Date).dt.month
spray_data['year']=pd.to_datetime(spray_data.Date).dt.year

#spray_data_polygon=spray_data_polygon.drop('Date', axis=1)
#spray_data_polygon=spray_data_polygon.drop('Time', axis=1)

train_data['month']=pd.to_datetime(train_data.Date).dt.month
train_data['year']=pd.to_datetime(train_data.Date).dt.year
train_data['RESTUANS']=train_data['Species'].str.contains('RESTUANS')
train_data['PIPIENS']=train_data['Species'].str.contains('PIPIENS')
#train_data[train_data['WnvPresent']>0]

In [235]:
def Prepear_coordinat(df):
    df_copy=df.copy()
    list_drop_column={'Latitude', 'Longitude'}

    for colomn in df_copy.columns:
        if colomn in list_drop_column:
            pass
        else:
            df_copy=df_copy.drop(colomn, axis=1)
                
    my_array = np.asarray (df_copy)
    
    return my_array

In [236]:
'''def Prepear_coordinat(df):
    df_copy=df.copy()
    list_drop_column={'Date', 'Time', 'month', 'year', 
                 'Address', 'Species', 'Block', 'Species', 'Block',
                'Street', 'Street', 'Trap', 'AddressNumberAndStreet',
                 'NumMosquitos', 'WnvPresent', 'AddressAccuracy', 'WnvPresent'}

    for colomn in df_copy.columns:
        if colomn in list_drop_column:
                df_copy=df_copy.drop(colomn, axis=1)
                
    my_array = np.asarray (df_copy)
    
    return my_array'''

def Add_shapesLayer_Polygon(df):
    if Prepear_coordinat(df).shape[0]>0:        
        return folium.Polygon(Prepear_coordinat(df),
                              color='red',
                              weight=4,
                              fill=True,
                              fill_color='green',
                              fill_opacity=0.6)

def numMosquitos(draft_year, draft_month):
    numMosquitos=train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                   & (geo_df['year']==draft_year)]['WnvPresent'].sum()
    return str(numMosquitos)    

In [237]:
#Spray data add to map
draft_year_list={2011, 2013}
draft_month_list={6, 7, 8, 9}

draft_year=2003

#spray_data_year=spray_data[spray_data['year']==2011]

map_ = folium.Map(location=[train_data['Latitude'].mean(),  train_data['Longitude'].mean()],
               tiles="Stamen Terrain", zoom_start=10, control_scale=False)

for draft_year in draft_year_list:
    spray_data_year=spray_data[spray_data['year']==draft_year].copy()
    draft_month=1
    if spray_data_year.shape[0]>0:
        for draft_month in draft_month_list:
            spray_data_month=spray_data_year[spray_data_year['month']==draft_month].copy()
            if spray_data_month.shape[0]>0:
                shapesLayer=folium.FeatureGroup(name=f'spray_{draft_year}_{draft_month}')
                shapesLayer.add_to(map_)
                Add_shapesLayer_Polygon(spray_data_month).add_to(shapesLayer)

            draft_month=draft_month+1
    
    draft_year=draft_year+1
#folium.LayerControl().add_to(map_)
#map_.save(f"spray_map.html")

In [238]:
# train_data_point_map
draft_year_list={2007, 2009, 2013}
draft_month_list={6, 7, 8, 9}

geometry = geopandas.points_from_xy(train_data.Longitude, train_data.Latitude)
geo_df = geopandas.GeoDataFrame(
    train_data, geometry=geometry
)
geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in geo_df.geometry]

wnvpresentsLayer=folium.FeatureGroup(name='wnvpresents')
wnvpresentsLayer.add_to(map_)

# Iterate through list and add a marker for each volcano, color-coded by its type.
i = 0
for coordinates in geo_df_list:
    if (geo_df.month[i] == draft_month) and  (geo_df.year[i] == draft_year):
        wnvpresents=int(train_data[(geo_df['Trap'] == geo_df.Trap[i]) & (geo_df['month']==draft_month) 
                   & (geo_df['year']==draft_year)]['WnvPresent'].sum())
        if wnvpresents>0:
            if geo_df.PIPIENS[i] == True:
                type_color = "green"
            if (geo_df.RESTUANS[i] == True):
                type_color = "red"
                # Place the markers with the popup labels and data
            wnvpresentsLayer.add_child(
                folium.Marker(
                    location=coordinates,
                    popup=geo_df.Trap[i]+ "<br>"
                    +'wnvpresents '+ str(wnvpresents)+ "<br>"+'numMosquitos '+ str(numMosquitos(draft_year, draft_month)),
                    icon=folium.Icon(color="%s" % type_color)
                    )
            )
    i = i + 1

In [239]:
# train_data_heat_map
draft_year_list={2007, 2009, 2013}
draft_month_list={6, 7, 8, 9}
for draft_year in draft_year_list:
    data_year=train_data[train_data['year']==draft_year].copy()
    
    for draft_month in draft_month_list:
        train_data_draft=train_data[(train_data['month']==draft_month)].copy()
        points = (train_data.Latitude.fillna(0),train_data.Longitude.fillna(0))
        coordinates =[]
        # Setting lat and long 
        lat = points[0]
        long = points[1]

    # Append latitude and longitude coordinates array
        for la,lo in zip(lat,long):
            coordinates.append([la,lo])

        train_heatLayer=folium.FeatureGroup(name=f'train_heat_map_{draft_year}_{draft_month}')
        train_heatLayer.add_to(map_)
        train_heatLayer.add_child(plugins.HeatMap(coordinates[0:20000]))
       
folium.LayerControl(position='topright', collapsed=False, autoZIndex=False).add_to(map_)
map_.save(f"train_heat_map.html") 